# TP6 - Clustering : Recommandations de livres

Dans cette séance on s'intéresse au dataset issue de [wonderbk.com](wonderbk.com) un site de vente de livre en ligne. Nous allons réaliser des clusters de livre pour constuire un système de recommandation de livre.
Les informations dont on dispose sont :
* **Title** : titre du livre
* **Authors** : Auteur du livre
* **Description** : Description du livre
* **Category** : Catégorie ou genre du livre
* **Publisher** : Maison d'édition du livre
* **Price Starting With ($)** : Prix initial du livre
* **Publish Date** : Date de la publication du livre

Commençons par importer les données.

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("BooksDataset.csv")
df.head()

## Nettoyage

On note que le dataset n'est pas exploitable en l'état, nous allons devoir faire un travail de mise en qualité avant de pouvoir construire le système de recommandation. Commençons par la colonne *Price*.

**Consigne** : Nettoyer la colonne *Price* pour la rendre numérique.

**Consigne** : Constuire la colonne *Publish Year* qui correspond à l'année de publication.

**Consigne** : Afficher la répartition du nombre de livre publié par an.

**Consigne** : Ne sélectionner que les livres publiés entre 1950 et 2024.

Il semblerait que la colonne *Authors* commence systématiquement par *By_*. 

**Consigne** : Vérifier si c'est effectivement le cas, et nettoyer la colonne en conséquence.

On s'intéresse avant de continuer aux valeurs manquantes. Pour construire notre système de recommandation, on souhaite en avoir aucune.

**Consigne** : Après avoir identifier avec la méthode [`isna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isna.html) les colonnes contenant des valeurs manquantes, les supprimer avec la méthode [`dropna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html).


On souhaite pouvoir exploiter la colonne *Category*. Pour le faire, nous allons faire un one hot encoding, mais nous devons le faire sur chaque categorie. Commençons par calculer la fréquence d'apparition de chaque catégorie.

In [ ]:
categories = {}

for row in range(df.shape[0]):
    if type(df["Category"][row]) != str:
        continue
    for category in df["Category"][row].split(" , "):
        category = category.strip()
        if category in categories.keys():
            categories[category] += 1
        else:
            categories[category] = 1

categories = dict(sorted(categories.items(), key=lambda x: x[1], reverse=True))
print(categories)


**Consigne** : Sélectionner les 20 premiers types les plus fréquent, puis créer autant de colonne valant 1 si le livre correspond à la catégorie, 0 sinon.

## Exploitation des descriptions

Beaucoup d'informations sont comprises dans ces lignes, et nous devons être capable de le transformer en nombre. Commençons par de l'analyse de sentiments. Il existe plusieurs librairies permettant de faire cela, la première est [TextBlob](https://textblob.readthedocs.io/en/dev/) :

In [ ]:
from textblob import TextBlob

texts = ["Theo always give great lecture", "Theo is reallty not a good lecturer", "Theo is OK at his job"]
for text in texts:
    blob = TextBlob(text)
    print(f"Analyse de \"{text}\" : {blob.sentiment}")

Une autre est [VADER](https://vadersentiment.readthedocs.io/en/latest/) :

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
for text in texts:
    scores = analyzer.polarity_scores(text)
    print(f"Analyse de \"{text}\" : {scores}")

On souhaite intégrer ces informations dans notre datasets.

**Consigne** : Créer une colonne *Description_sentiment_Blob* qui correspond à la valeur *polarity* calculée par TextBlob et une autre colonne *Description_sentiment_VADER* qui correspond à la valeur *compound* calculée par VADER. Faire de même avec la colonne *Title*.

On souhaite étudier la distribution et la corrélation entre les scores, pour le titre et la description des livres.

**Consigne** : Afficher sur un même graphique la distribution des scores pour la description entre TextBlob et VADER.

**Consigne** : Afficher un scatter plot entre les score de TextBlob et de VADER pour les deux colonnes concernées.

## Calcul des clusters

Nous avons maintenant accès à un dataset permettant d'exploiter du Machine Learning non supervisé. Puisque les résolutions des problèmes que les algorithmes vont résoudre peuvent exploiter des variantes d'une descente de gradient, nous devons préparer en conséquence la matrice.

**Consigne** : A partir du dataset *df*, ne conserver que les colonnes numériques puis avec la classe [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) standardiser les données.

**Consigne** : A l'aide de la classe [`KMeans`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) calculer les clusters et les stocker dans une variable *labels*. On prendra *n_clusters=5*. Finalement, afficher le nombre d'observations dans chaque cluster.

**Consigne** : Calculer la performance du clustering avec la fonction [`silhouette_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html)

On souhaite a présent avoir un peu d'explicabilité pour ces clusters. Pour le faire, nous allons reporter les labels appris dans le dataset initial.

**Consigne** : Construire un dataset *data* à partir de *df*, et y ajouter le vecteur *labels*. On supprimera des catégories OHE pour la lisibilité.

**Consigne** : Explorer un des clusters et essayer de comprendre la constitution du cluster. On pourra utiliser la méthode `describe` pour les données numériques.

Testons à présent l'algorithme DBSCAN. Nous n'avons pas cette fois à spécifier le nombre de cluster que l'on souhaite obtenir. Nous devons donc commencer par identifier les informations que l'on souhaite avoir.

**Consigne** : Définir une fonction `explore_clusters` qui prend en paramètre la matrice $X$ et les labels calculés. Elle affichera :
* Le silhouette score
* Le nombre de cluster
* La répartition des observations dans les clusters

**Consigne**: Entraîner avec les valeurs par défaut l'algorithme [`DBSCAN`](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN) puis utiliser la fonction `explore_clusters`.

**Consigne**: A l'aide du cours et par tatônnement entraîner DBSCAN avec d'autres paramètrages.